In [2]:
import gym_trading_env
from gym_trading_env.downloader import download, EXCHANGE_LIMIT_RATES
#import gymnasium as gym # No longer used and is installed with other libraries anyways
import gymnasium as gym
import datetime
import pandas as pd
import numpy as np
import os # used to get current directory
from tqdm import tqdm
# Import all the Pearl related methods
from pearl.pearl_agent import PearlAgent
from pearl.action_representation_modules.one_hot_action_representation_module import (
    OneHotActionTensorRepresentationModule,
)
from pearl.policy_learners.sequential_decision_making.deep_q_learning import (
    DeepQLearning,
)
from pearl.replay_buffers.sequential_decision_making.fifo_off_policy_replay_buffer import (
    FIFOOffPolicyReplayBuffer,
)
from pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from pearl.policy_learners.sequential_decision_making.implicit_q_learning import ImplicitQLearning
from pearl.utils.functional_utils.train_and_eval.offline_learning_and_evaluation import (
    get_offline_data_in_buffer,
    offline_evaluation,
    offline_learning,
)
from pearl.neural_networks.sequential_decision_making.actor_networks import VanillaContinuousActorNetwork
from pearl.pearl_agent import PearlAgent
from pearl.action_representation_modules.one_hot_action_representation_module import (
    OneHotActionTensorRepresentationModule,
)
from pearl.policy_learners.sequential_decision_making.deep_q_learning import (
    DeepQLearning,
)
from pearl.replay_buffers.sequential_decision_making.fifo_off_policy_replay_buffer import (
    FIFOOffPolicyReplayBuffer,
)
from pearl.utils.instantiations.environments.gym_environment import GymEnvironment
import torch # used for calculating the mean when evaluating Pearl training performance

##FOR USE WITH VANILLAQVALUENETWORK
#from pearl.neural_networks.sequential_decision_making.q_value_networks import VanillaQValueNetwork

experiment_seed = 42

In [3]:
## reward funtions

# Define reward function here. Current function is based on last 2 timesteps (one hour apart)
def log_reward_function(history):
        return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2])

def diff_reward_function(history):
        return history["portfolio_valuation", -1] - history["portfolio_valuation", -2]
def cumulative_reward_function(history):
        return np.diff(history["portfolio_valuation", -1] , history["portfolio_valuation", 0])/len(history["portfolio_valuation"])

def shape_reward_function(history):
        return history["portfolio_valuation", -1] /np.std(history["portfolio_valuation"])


In [9]:
positions = [
    [ -1, 0, 1],
    [-.5,0,.5],
    [-1,-.5,0,.5,1],
    [-1,1],
    ]
reward_functions=[shape_reward_function,
                  log_reward_function,
                  diff_reward_function,
                  cumulative_reward_function,
                  ]
hidden_dims=[[64, 64],[128,128],[256,256]]

num_layers=[2,3,4]
num_dims=64
training_rounds=20

def make_arch(num_layers,num_dims):
    return[num_dims for n in range(num_layers)]

In [10]:
make_arch(num_layers=num_layers,num_dims=num_dims)

[64, 64]

In [11]:
from hyperopt import hp


In [16]:
str(1e-5)

'1e-05'

In [ ]:
space={
    'num_layers':hp.choice('num_layers',num_layers),
    'reward_function':hp.choice(reward_functions),
    'learning_rate':hp.loguniform('learning_rate',1e-5,1e-2)
    
}

In [ ]:
num_actions = train_env.action_space.n
agent = PearlAgent(
    policy_learner=DeepQLearning(
        state_dim=train_env.observation_space.shape[0],
        action_space=train_env.action_space, 
        hidden_dims=[64, 64],
        
        training_rounds=20,
        action_representation_module=OneHotActionTensorRepresentationModule(
            max_number_actions=num_actions
        ),
    ),
    replay_buffer=FIFOOffPolicyReplayBuffer(10_000),
)

In [ ]:
def make_env(positon,reward_function):
    test_env = gym.make("TradingEnv",
        name= "ICPUSD",
        df = test_df,
        positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
        reward_function = reward_function,
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
    )

In [ ]:
def objective_train_agent(gamma,lr,hidden_dems,num_episodes,train_after_episode):
    agent=make_agent(gamma,lr,hidden_dems)
    

    test_env

    return profit